# Assignment 1

## Import libaruaries may need

In [1]:
from bs4 import BeautifulSoup
import requests
import re

import pandas as pd
import numpy as np

## Save data to local for convieniont

In [2]:
def curl(fname, path, fmode):
    """
    copy url=path, save as fname, read and write as 'wb' mode
    """
    import urllib.request, urllib.parse, urllib.error
    
    response = urllib.request.urlopen(path).read()
    with open(fname, fmode) as fhand:
        fhand.write(response)

## Parse data to BeautifulSoup

In [3]:
fname = 'postal_codes_of_Canada.html'

# path = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# curl(fname, path, 'wb')

with open(fname,'rb') as html_file:
    soup = BeautifulSoup(html_file, 'lxml') 

## Find the table data and store it in np.array

In [4]:
table = soup.find('table', class_="wikitable sortable")
# print(table.find_all('td'))

tds = np.array([])
for td in table.find_all('td'):
    tds = np.append(tds, td.text.rstrip(), axis=None)
print(tds[0:6])
len(tds)

['M1A' 'Not assigned' 'Not assigned' 'M2A' 'Not assigned' 'Not assigned']


864

## Find the table headers and store it in a np.array

In [5]:
ths = np.array([])
for th in table.find_all('th'):
    ths = np.append(ths, th.text.rstrip(), axis=None)
print(ths)

['Postcode' 'Borough' 'Neighbourhood']


## Reshape the np.array to 3 columns, and add table headers

In [6]:
pc_df = pd.DataFrame(np.reshape(tds, (int(len(tds)/3),3)), columns=ths)
pc_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Replace 'Not assigned' in ['Bourough'] with np.nan, and drop it.

In [7]:
pc_df['Borough'].replace("Not assigned", np.nan, inplace = True)
pc_df.dropna(subset=['Borough'],axis=0, inplace=True)
pc_df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Copy a data frame in case of fatal

In [22]:
pc_df1 = pc_df
pc_df1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Goup by ['Postcode','Borough'], and join ['Neighbourhood'] with ','

In [23]:
pc_df1 = pc_df1.groupby(['Postcode','Borough']).apply(lambda x : ','.join(x['Neighbourhood']))
pc_df1 = pd.DataFrame(pc_df1)
pc_df1 = pc_df1.reset_index()
pc_df1.head()

,Postcode,Borough,0
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Rename the new column

In [24]:
pc_df1.rename(columns={0:'Neighbourhood'}, inplace=True)
pc_df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Replace ['Neighbourhood'] where value is "Not assigned" equal to ['Borough']

In [25]:
pc_df1['Neighbourhood'].replace("Not assigned", pc_df1['Borough'], inplace=True)
pc_df1[(pc_df1['Borough']==pc_df1['Neighbourhood'])]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


## Assignment 1 finished, store it to df_clean, and check the shape.

In [28]:
df_clean = pc_df1
df_clean.shape

(103, 3)

In [29]:
df_clean.to_csv('Pc_Canada_clean.csv', index=None)